In [2]:
import importlib
import sys
from tqdm import tqdm
import  os
import pickle
import glob
from datetime import datetime
import matplotlib.pyplot as plt
# Add the directories to the sys.path
dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "tools")

import Analysis_tools as ATools
import Atlas_tools as AtTools
import proAnubis_Analysis_Tools
import Reconstruction_tools as RTools
import numpy as np
import mplhep as hep
import Timing_tools as TTools
import gc
import pickle
import overview
import rawFileReader
import Visual_tools as VTools
hep.style.use([hep.style.ATLAS])

dir = "C://Users//jony//Programming//Python//Anubis//anubis//data//"

# Specify the directory


In [3]:
importlib.reload(AtTools)
analyser = AtTools.AtlasAnalyser()
start = "2024-09-05 05:58:04" 
end = "2024-09-05 06:40:23"
file_name = "proAnubis_240905_0635.raw"

analyser.get_tdc5_data(file_name, 1, 2, 100)
initial_time = analyser.anubis_data[0].event_time
print(initial_time)
print(datetime.timestamp(initial_time))
event_time = initial_time

start = datetime.strptime(start, '%Y-%m-%d %H:%M:%S')
end = datetime.strptime(end, '%Y-%m-%d %H:%M:%S')
previous_event_time = initial_time
with tqdm(total=round((start-initial_time).total_seconds()), desc=f"Skipping {file_name}", unit='tdc5 Reads') as pbar:
    while event_time < start:
        analyser.fReader.skip_events(20_000)
        event_time = None
        while not event_time:
            if not analyser.fReader.readBlock():
                    raise EOFError("You have reached the end of the file")
            if analyser.fReader.hasEvents():
                tdc5Reads = analyser.fReader.getTDCFiveEvents()
                if not tdc5Reads:
                    continue
                else:
                    event_time = tdc5Reads[0][0]
                    pbar.update(event_time.timestamp() - previous_event_time.timestamp())
                    previous_event_time = event_time
previous_event_time = start
print(event_time)
with tqdm(total=round((end-start).total_seconds()), desc=f"Reading {file_name}", unit='seconds') as pbar:        
    while event_time < end:
        gc.disable()
        analyser.anubis_data = []
        try:
            analyser.get_tdc5_data(file_name, 1, 2, amount_of_events=20_000)
        except EOFError:
            file_name = "proAnubis_240905_0635.raw"
    
        event_time = analyser.anubis_data[-1].event_time
        print("Event time:", event_time)
        print("Timestamp:", analyser.anubis_data[-1].timeStamp)
        pbar.update(round(event_time.timestamp()- previous_event_time.timestamp()))
        previous_event_time = event_time
        with open(dir + f"correct_bcr//{round(event_time.timestamp())}", "wb") as f:
            pickle.dump(analyser.anubis_data, f)
        gc.enable()
        gc.collect()
        

            

New
2024-09-05 05:35:00.028855
1725510900.028855


Skipping proAnubis_240905_0635.raw: 1386.8695359230042tdc5 Reads [01:48, 12.77tdc5 Reads/s]                  


2024-09-05 05:58:06.898391


Reading proAnubis_240905_0635.raw:   0%|          | 3/2539 [00:13<3:14:56,  4.61s/seconds]

Event time: 2024-09-05 05:58:06.901083
Timestamp: 1725512286.901083


...

Event time: 2024-09-05 06:07:35.385442
Timestamp: 1725512855.385442


Reading proAnubis_240905_0635.raw:  25%|██▍       | 624/2539 [06:31<26:30,  1.20seconds/s]

Event time: 2024-09-05 06:08:27.779386
Timestamp: 1725512907.779386


Reading proAnubis_240905_0635.raw:  27%|██▋       | 677/2539 [10:39<1:02:14,  2.01s/seconds]

Event time: 2024-09-05 06:09:20.318751
Timestamp: 1725512960.318751


In [42]:
importlib.reload(AtTools)

analyser = AtTools.AtlasAnalyser() # 784_480
data_list = sorted([f for f in os.listdir("data//correct_bcr") if os.path.isfile(os.path.join("data//correct_bcr", f))], key=lambda x: int(x[3:-4]))
rates = []
times = []
with tqdm(total=len(data_list)) as pbar:
    for data in data_list:
        gc.disable()
        with open(f"data//correct_bcr//{data}", "rb") as f:
            analyser.anubis_data += pickle.load(f)
        trigger_rates, trigger_times = analyser.trigger_rates()
        rates.extend(trigger_rates)
        times.extend(trigger_times)
        analyser.anubis_data = []
        pbar.update(1)
        gc.enable()
        gc.collect()
print(len(rates)) 


 76%|███████▌  | 25/33 [01:08<00:21,  2.73s/it]


KeyboardInterrupt: 

In [43]:
importlib.reload(AtTools)

analyser = AtTools.AtlasAnalyser() # 784_480
data_list = sorted([f for f in os.listdir("data//correct_bcr") if os.path.isfile(os.path.join("data//correct_bcr", f))], key=lambda x: int(x[3:-4]))
with tqdm(total=len(data_list)) as pbar:
    for data in data_list:
        gc.disable()
        with open(f"data//correct_bcr//{data}", "rb") as f:
            analyser.anubis_data += pickle.load(f)
        pbar.update(1)
        gc.enable()
        gc.collect()
print(len(rates)) 


  0%|          | 0/33 [00:00<?, ?it/s]

100%|██████████| 33/33 [11:11<00:00, 20.33s/it]

75


In [45]:
data = analyser.anubis_data.copy()

In [46]:
importlib.reload(AtTools)
analyser = AtTools.AtlasAnalyser()
analyser.anubis_data = data.copy()

In [ ]:
#generate random data
importlib.reload(ATools)
data = AtTools.generate_random_triggers(1_000_000)
hist, bins = np.histogram(data, bins=list(range(0, 3654)), density=True)
plt.plot(bins[:-1], hist)
plt.title("Random triggers")
plt.xlabel("Bunch Crossings [bx]")
plt.ylabel("Frequency")
plt.show()

In [47]:
#Atlas load
importlib.reload(AtTools)
atlas_file = "C://Users//jony//Programming//Python//Anubis//anubis//data//dataAtlas2.root"
analyser.get_atlas_data(atlas_file)
print("Loading done")

Loading done


In [51]:
#Basic analysis
print("Anubis len:", len(analyser.anubis_data))
print("ATLAS len:", len(analyser.atlas_data))
print("Atlas first:",analyser.atlas_data.iloc[0]["TimeStamp"])
print("Anubis first:",analyser.anubis_data[0].timeStamp)
print("Atlas last:", analyser.atlas_data.iloc[-1]["TimeStamp"])
print("Anubis last:", analyser.anubis_data[-1].timeStamp)

Anubis len: 28190636
ATLAS len: 942478
Atlas first: 1725509842
Anubis first: 1725511462.535904
Atlas last: 1725528310
Anubis last: 1725513012.834854


In [56]:
#Trigger rates
trigger_rates, times = analyser.trigger_rates()
#hist, bins = analyser.beam_luminosity()
#hist = max(trigger_rates)/max(hist)*hist

plt.plot(bins[:-1], hist, label="Luminosity ATLAS")
plt.step(times, trigger_rates, where="post", label="Trigger rates")
plt.legend()
plt.xlabel("TimeStamp")
plt.xlim([times[0], times[-1]])
plt.show()


In [ ]:
#Same times
analyser.atlas_data = analyser.atlas_data[(analyser.anubis_data[0].timeStamp < analyser.atlas_data["TimeStamp"]) & (analyser.atlas_data["TimeStamp"] < analyser.anubis_data[-1].timeStamp)]

In [41]:
#hist, bins = np.histogram(times, bins = np.arange(min(times), max(times), 10), density=True)
fake_rates = [r*0.00000001 for r in rates]
plt.step(times, fake_rates, where="post", label = "proAnubis")
#plt.title(f"Luminosity for {start}-{end}")
"""
print(analyser.atlas_data["TimeStamp"][0])
analyser.atlas_data["TimeStamp"] += 3600
#print(analyser.atlas_data["TimeStamp"][0])
"""
tim = analyser.atlas_data["TimeStamp"]
hist, bins = np.histogram(tim, bins=1000, density=True)
plt.plot(bins[:-1], hist, label="Atlas")
plt.legend()
#plt.xlim(1719500000, 1719506000)
plt.show()

In [21]:
importlib.reload(AtTools)
AtTools.beam_luminosity(analyser.atlas_data)

In [4]:
sixteen_twenty = trig.copy()

In [21]:
complete = sixteen_twenty+trig[2:]

start = "2024-06-27 14:20:28" 
end = "2024-06-27 17:40:23"
hist, bins = np.histogram(complete, bins = np.arange(min(complete), max(complete), 10))
plt.plot(bins[1:-1], 10*hist[1:])
plt.title(f"'Event Rate' for proAnubis {start}-{end}")
plt.show()

In [13]:
AtTools.plot_offset(analyser)

Best offset 114


In [26]:
print(len(analyser.anubis_data))

23024320


In [32]:
analyser.match_bcrs() #34705

Matching: 100%|██████████| 69601/69601 [02:09<00:00, 535.54Events/s]


[[{'RunNumber': 484126,
   'EventNumber': 0,
   'LumiBlock': 236,
   'TimeStamp': 1725511708,
   'TimeStampNS': 3629985,
   'BCID': 902,
   'extL1ID': 3674536648,
   'triggerType': 132,
   'mu_pt': <Array [2.21] type='1 * float32'>,
   'mu_eta': <Array [1.94] type='1 * float32'>,
   'mu_phi': <Array [1.83] type='1 * float32'>,
   'mu_E': <Array [7.88] type='1 * float32'>,
   'mu_charge': <Array [-1] type='1 * int32'>,
   'jet_pt': <Array [43.4, 41.6, 12.6, 11.4, ..., 7.5, 7.35, 7.14, 7.02] type='15 * float32'>,
   'jet_eta': <Array [0.826, 0.207, -0.371, ..., -2.31, 0.22, -0.625] type='15 * float32'>,
   'jet_phi': <Array [2.4, -0.603, -0.937, ..., 0.316, 0.827, -1.44] type='15 * float32'>},
  []],
 [{'RunNumber': 484126,
   'EventNumber': 0,
   'LumiBlock': 236,
   'TimeStamp': 1725511708,
   'TimeStampNS': 5131470,
   'BCID': 492,
   'extL1ID': 3674536798,
   'triggerType': 132,
   'mu_pt': <Array [] type='0 * float32'>,
   'mu_eta': <Array [] type='0 * float32'>,
   'mu_phi': <Array

In [33]:
print(len([1 for i in analyser.matches if i[1]]))

1659


In [34]:
importlib.reload(AtTools)
times, eta, phi, heatmap = AtTools.convert_matches(analyser.matches, 114)

In [29]:
import background

bg = background.calculate_background(anubis_data=analyser.anubis_data)


100%|██████████| 3564/3564 [00:02<00:00, 1507.49it/s]


In [35]:
#Position

plt.hist2d(eta, phi, bins=30, cmap='plasma')
plt.xlim(-2.5, 2.5)
plt.ylim(-np.pi, np.pi)
plt.xlabel("eta")
plt.ylabel("phi")
plt.colorbar()
plt.show()

In [15]:
# position 3d
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
hist, xedges, yedges = np.histogram2d(eta, phi, bins=30, range=[[-np.pi, np.pi], [-np.pi, np.pi]])

xpos, ypos = np.meshgrid(xedges[:-1] + 0.5 * (xedges[1] - xedges[0]),
                         yedges[:-1] + 0.5 * (yedges[1] - yedges[0]),
                         indexing="ij")

# Initialize the 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot the surface
ax.plot_surface(xpos, ypos, hist, cmap='plasma')

# Set labels
ax.set_xlabel('eta')
ax.set_ylabel('phi')
ax.set_zlabel('Count')

# Set limits
ax.set_xlim(-np.pi, np.pi)
ax.set_ylim(-np.pi, np.pi)
plt.show()


In [12]:
#times
plt.hist(times, bins=range(min(times), max(times) + 2, 1), density=True)
#plt.plot(range(0, 3564), bg)
#plt.xlim(-100,100)
plt.xlabel("Time difference (bx)")
#plt.xlim(-500,500)
plt.ylabel("Frequency")
plt.title("Time difference between BCR and trigger")
plt.show()

In [19]:
print(len([1 for i in times if i == 3]))
print(len(analyser.anubis_data))
print(len(analyser.matches))

56036
33653567
28813
143415761


NameError: name 'bg' is not defined

In [57]:
#compute correlation from heatmap
atlas = []#AtTools.generate_random_triggers(10_000_000)
anubis = []#AtTools.generate_random_triggers(10_000_000)

for i in range(len(heatmap)):
    for j in range(len(heatmap[i])):
        for count in range(heatmap[i][j]):
            atlas.append(i)
            anubis.append(j)

from scipy.stats import pearsonr
r, p = pearsonr(atlas, anubis)
print("Pearson coefficient:", r)
print("p-value:", p)


Pearson coefficient: 0.002179741957029581
p-value: 0.32391912641600673


In [54]:
print(len(atlas))

204801


In [55]:
#histogram of the matched bcrs
anubis = []
for match in analyser.matches:
    if match[1] != []:
        anubis.extend(match[1])
hist = []
for trigger in anubis:
    hist.append(round(trigger.bcId))

bins = [i-0.5 for i in range(0,3565)] #-0.5,0.5,...3653.5
counts, _ = np.histogram(hist, bins=bins, density=False)
bins = [i for i in range(0,3564)]
plt.plot(bins, counts, color="orange")

plt.xlabel('Time since last BCR (ns)')
plt.xlim(0, 3565)
plt.ylabel('Frequency')
plt.title(f'Histogram of BCID Anubis')
plt.show()

In [43]:
#heatmap
plt.imshow(heatmap)
plt.plot([i for i in range(3564)],[i for i in range(3564)], color = "red")
plt.xlim(0, 3564)
plt.ylim(0, 3564)
#plt.gca().invert_yaxis()
plt.xlabel("Atlas BX")
plt.ylabel("proANUBIS BX")
plt.title("Heatmap of BX matches")
plt.show()